In [11]:
# Api used jupyter notebook
# Genre - genre_id, genre_name   (formally subcategories)
# Event_Capacity - event_id, quantity_sold, quantity_pending, quantity_total
# Event- event id, summary, category_id, event_name, start_date, end_date, URL, status?, online, age_restriction?, 
# Ticket_Classes - event_id, description, capacity, quantity_sold
# Venue - event_id, venue_id, only_public, status, venue_name, age_restriction, city, region, postal_code, (latitude, longitude for the map)

In [12]:
#imports
import pandas as pd
import requests 

path = ('Project03_Visualization_SHOWWW\csv_tables')

In [13]:
#base url
#url =  'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&classificationName=Music&startDateTime=2023'
genre_url = 'https://app.ticketmaster.com/discovery/v2/classifications/KZFzniwnSyZfZ7v7nJ.json?apikey=yTIm5zlYHTsXuGFluFTMc9u5peB89aGA'

In [14]:
#getting genre from ticketmaster developer
# using separate api pull for simple list of music genres

genre_call = requests.get(genre_url)
genre_call_json = genre_call.json()
# print(genre_json)
subgenre_json = genre_call_json['segment']['_embedded']['genres'][4]['_embedded']['subgenres']
genre_json = genre_call_json['segment']['_embedded']['genres']
# print(subgenre_json)
# print(genre_json)

genre_list = []
subgenre_list = []

for genre in genre_json:
    genre_id = genre['id']
    genre_name = genre['name']
    subgenres = genre['_embedded']['subgenres']

    for subgenre in subgenres:
        subgenre_name = subgenre['name']
        subgenre_id = subgenre['id']
        genre_list.append({'genre': genre_name, 'id': genre_id, 'subgenre': subgenre_name, 'subgenre_id': subgenre_id})
        subgenre_list.append(subgenre_name)

subgenre_list
genre_list




[{'genre': 'Alternative',
  'id': 'KnvZfZ7vAvv',
  'subgenre': 'Adult Alternative Pop/Rock',
  'subgenre_id': 'KZazBEonSMnZfZ7vAvE'},
 {'genre': 'Alternative',
  'id': 'KnvZfZ7vAvv',
  'subgenre': 'Alternative',
  'subgenre_id': 'KZazBEonSMnZfZ7vAvJ'},
 {'genre': 'Alternative',
  'id': 'KnvZfZ7vAvv',
  'subgenre': 'Alternative Dance',
  'subgenre_id': 'KZazBEonSMnZfZ7vAvI'},
 {'genre': 'Alternative',
  'id': 'KnvZfZ7vAvv',
  'subgenre': 'Alternative Metal',
  'subgenre_id': 'KZazBEonSMnZfZ7vAvt'},
 {'genre': 'Alternative',
  'id': 'KnvZfZ7vAvv',
  'subgenre': 'Alternative Rock',
  'subgenre_id': 'KZazBEonSMnZfZ7vAvn'},
 {'genre': 'Alternative',
  'id': 'KnvZfZ7vAvv',
  'subgenre': 'Ambient Pop',
  'subgenre_id': 'KZazBEonSMnZfZ7vAvl'},
 {'genre': 'Alternative',
  'id': 'KnvZfZ7vAvv',
  'subgenre': 'American Alternative',
  'subgenre_id': 'KZazBEonSMnZfZ7vAev'},
 {'genre': 'Alternative',
  'id': 'KnvZfZ7vAvv',
  'subgenre': 'Americana',
  'subgenre_id': 'KZazBEonSMnZfZ7vAee'},
 {'genre'

In [24]:
genre_df = pd.json_normalize(genre_list)
genre_df = genre_df.rename(columns={'id':'genre_id','genre':'genre_name'})
genre_df = genre_df.set_index('genre_id')
genre_df.to_csv('csv_tables/genre.csv')
genre_df

,genre_name,subgenre,subgenre_id
genre_id,,,
KnvZfZ7vAvv,Alternative,Adult Alternative Pop/Rock,KZazBEonSMnZfZ7vAvE
KnvZfZ7vAvv,Alternative,Alternative,KZazBEonSMnZfZ7vAvJ
KnvZfZ7vAvv,Alternative,Alternative Dance,KZazBEonSMnZfZ7vAvI
KnvZfZ7vAvv,Alternative,Alternative Metal,KZazBEonSMnZfZ7vAvt
KnvZfZ7vAvv,Alternative,Alternative Rock,KZazBEonSMnZfZ7vAvn
...,...,...,...
KnvZfZ7vAeF,World,Worldbeat,KZazBEonSMnZfZ7vFan
KnvZfZ7vAeF,World,Zaire & Congo,KZazBEonSMnZfZ7vFal
KnvZfZ7vAeF,World,Zimbabwe,KZazBEonSMnZfZ7vF1v


In [16]:
base_url =  'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&classificationName=Music&startDateTime=2023-06-20T00:00:00Z&sort=date,asc&size=200&dmaId=324&localStartDateTime:2023-06-20T00:00:00Z,2023-07-20T00:00:00Z&apikey=yTIm5zlYHTsXuGFluFTMc9u5peB89aGA'

event_call = requests.get(base_url)
event_call_json = event_call.json()

event_json = event_call_json['_embedded']['events']

event_data = []
genre_id = []
subgenre_id = []
seatMapUrl_list = []
venueURL_list = []
venue_data = []

for event in event_json:
    event_name = event['name']
    event_id = event['id']
    event_url = event['url']
    image_url = event['images'][0]['url']
    startDate = event['dates']['start']['localDate']
    startTime = event['dates']['start']
    localTime = startTime.get('localTime')
    classifications = event['classifications']
    venues = event["_embedded"]['venues']

    priceRange = event.get('priceRanges')
    min_price = None
    max_price = None
    
    if priceRange:
        min_price = priceRange[0].get('min')
        max_price = priceRange[0].get('max')
   
    for venue in venues:
        venue_name = venue['name']
        venue_id = venue['id']
        venue_postal = venue['postalCode']
        venue_city = venue['city']
        venue_state = venue['state']['stateCode']
        venue_add = venue['address']
        venue_lat = venue['location']['latitude']
        venue_long = venue['location']['longitude']
        venue_url = venue.get('url', '')  # Use .get() with a default value if 'url' key is missing
        venueURL_list.append({"Venue ID" : venue_id,
                              "Venue URL" : venue_url})
        venue_data.append({"Venue Name": venue_name, 
                           "Venue ID": venue_id, 
                           "Postal Code" : venue_postal, 
                           "City" : venue_city, 
                           "State" : venue_state,
                           "Address" : venue_add,
                           "Latitude" : venue_lat,
                           "Longitude" : venue_long
                           })
    
    seatMap = event.get('seatmap')
    if seatMap:
        seatMapUrl_list.append({'Venue ID' : venue_id,
                                'Seatmap' : seatMap['staticUrl']})
    
    for classification in classifications:
        genre = classification['genre']
        subgenre = classification.get('subGenre', '')  # Use .get() to safely retrieve the value
        genre_id = genre.get('id', '')
        if subgenre:
            subgenre_id = subgenre.get('id', '')

    event_data.append({"Event ID" : event_id,
                       "Event Name" : event_name,
                       "Venue ID" : venue_id,
                       "Event URL" : event_url,
                       "Image URL" : image_url,
                       "Start Date" : startDate,
                       "Start Time" : localTime,
                       "Genre ID" : genre_id,
                       "Subgenre ID" : subgenre_id,
                       "Minimum Price" : min_price,
                       "Maximum Price" : max_price
                       })
                       


In [17]:
if seatMap:
        seatMapUrl_list.append({'Venue ID' : venue_id,
                                'Seatmap' : seatMap['staticUrl']})

seatmap_df = pd.DataFrame(seatMapUrl_list)
seatmap_df = seatmap_df.rename(columns={'Venue ID': 'venue_id', 'Seatmap': 'SeatMap'})
seatmap_df = seatmap_df.set_index('venue_id')
seatmap_df.to_csv('csv_tables/seatmap.csv')
seatmap_df

,SeatMap
venue_id,
KovZpZAJvttA,https://maps.ticketmaster.com/maps/geometry/3/...
KovZ917AGff,https://maps.ticketmaster.com/maps/geometry/3/...
KovZ917AJ5H,https://maps.ticketmaster.com/maps/geometry/3/...
KovZpa6OCe,https://maps.ticketmaster.com/maps/geometry/3/...
KovZpa2Hxe,https://maps.ticketmaster.com/maps/geometry/3/...
...,...
KovZ917A8gf,https://maps.ticketmaster.com/maps/geometry/3/...
KovZ917AGff,https://maps.ticketmaster.com/maps/geometry/3/...
KovZpZA1vvlA,https://maps.ticketmaster.com/maps/geometry/3/...


In [18]:
event_df = pd.DataFrame(event_data)
event_df = event_df.set_index('Event ID')
event_df = event_df.rename(columns={'Event ID':'event_id','Event Name':'event_name','Venue ID':'venue_id','Event URL':'event_url',
                                      'Image URL':'image_url','Start Date':'startDate','Start Time':'startTime','Genre ID':'genre_id',
                                      'Subgenre ID':'subgenre_id','Minimum Price':'min_price','Maximum Price':'max_price'})
event_df.to_csv('csv_tables/events.csv')

event_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, vvG10Z9E9dt3nY to vvG10Z9lnWNdtF
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   event_name   200 non-null    object 
 1   venue_id     200 non-null    object 
 2   event_url    200 non-null    object 
 3   image_url    200 non-null    object 
 4   startDate    200 non-null    object 
 5   startTime    199 non-null    object 
 6   genre_id     200 non-null    object 
 7   subgenre_id  200 non-null    object 
 8   min_price    138 non-null    float64
 9   max_price    138 non-null    float64
dtypes: float64(2), object(8)
memory usage: 17.2+ KB


In [21]:

venue_df = pd.DataFrame(venue_data)
venue_df = venue_df.rename(columns={'Venue ID':'venue_id','Venue Name':'venue_name','Postal Code':'postalCode','City':'city','State':'state',
                                    'Address':'address','Latitude':'latitude','Longitude':'longitude'})
venue_df = venue_df.drop_duplicates(subset=['venue_id'])
venue_df = venue_df.set_index('venue_id')

venue_df.to_csv('csv_tables/venues.csv')

#venue_df = venue_df.drop_duplicates(subset=['venue_id'])
venue_df

,venue_name,postalCode,city,state,address,latitude,longitude
venue_id,,,,,,,
KovZpZAJvttA,The Belasco,90015,{'name': 'Los Angeles'},CA,{'line1': '1050 South Hill Street'},34.04024506,-118.2592392
KovZ917AGff,Constellation Room,92704,{'name': 'Santa Ana'},CA,{'line1': '3503 South Harbor Boulevard'},33.699879,-117.917493
KovZ917AJ5H,The Moroccan Lounge,90012,{'name': 'Los Angeles'},CA,{'line1': '901 E 1st Street'},34.04898,-118.23365
KovZpZAkldvA,Copper Blues Oxnard Resturant,93036,{'name': 'Oxnard'},CA,{'line1': '581 Collections Blvd'},34.240229,-119.176274
KovZpa2Hxe,Greek Theatre,90027,{'name': 'Los Angeles'},CA,{'line1': '2700 N. Vermont Ave'},34.11948811,-118.29629093
KovZpa6OCe,Echoplex,90026,{'name': 'Los Angeles'},CA,{'line1': '1154 Glendale Blvd'},34.0775847,-118.2606316
KovZpZAEA67A,House of Blues Anaheim,92802,{'name': 'Anaheim'},CA,"{'line1': '400 W Disney Way, #337'}",33.8095868,-117.9229975
KovZpZAEAl6A,The Wiltern,90010,{'name': 'Los Angeles'},CA,{'line1': '3790 Wilshire Blvd.'},34.06141494,-118.30879207
KovZ917AGEf,The Parish at House of Blues Anaheim,92802,{'name': 'Anaheim'},CA,"{'line1': '400 W Disney Way, #337'}",33.806533,-117.912474
